# Model Context Protocol (MCP) Integration with Azure AI Foundry Agents

## Introduction

This notebook demonstrates integrating **Model Context Protocol (MCP)** servers with Azure AI Foundry agents. MCP is an open protocol enabling AI applications to connect to data sources, tools, and services.

### What is MCP?

MCP standardizes how AI models:
- Access contextual information from various data sources
- Execute tools through standardized interfaces
- Connect securely to services and APIs

### Key Concepts

**MCP Server:**
- Exposes resources, tools, and prompts
- Uses JSON-RPC protocol
- Examples: File system, databases, APIs

**MCP Client:**
- Connects to MCP servers
- Discovers and invokes tools
- Your Azure AI application

**Transport:**
- **SSE (Server-Sent Events)**: HTTP-based, remote servers
- **stdio**: Local processes (advanced)

### Architecture

```
Azure AI Agent (GPT-4o)
    ↓
Your Application (MCP Client)
    ↓
MCP Servers (Tools & Data)
```

### Integration Flow

1. Initialize MCP tool with `McpTool` class
2. Create agent with `mcp_tool.definitions`
3. Run with `mcp_tool.resources`
4. SDK handles tool execution automatically

### Prerequisites

- Azure AI Foundry project with endpoint
- Python 3.10+
- Environment variables configured

## Table of Contents

1. [Part 1: Environment Setup](#part-1-environment-setup)
   - 1.1: Install Dependencies
   - 1.2: Configure Environment & Imports
   - 1.3: Initialize Azure AI Client
2. [Part 2: SSE MCP Server Integration](#part-2-sse-mcp-server-integration)
3. [Part 3: Complete MCP Integration Example](#part-3-complete-mcp-integration-example)
   - Step 1: Initialize MCP Tool
   - Step 2: Create Agent with MCP Tool
   - Step 3: Create Thread and Post Message
   - Step 4: Run Agent with MCP Tool Approval
   - Step 5: Read Assistant Response
4. [Summary and Best Practices](#summary-and-best-practices)

---

## Part 1: Environment Setup

### 1.1: Install Dependencies

Install required packages for MCP integration.

In [ ]:
%pip install -qU mcp  # Model Context Protocol SDK
%pip install -qU httpx  # For SSE transport

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

### 1.2: Configure Environment & Imports

Load environment variables and import necessary modules.

**Required Environment Variables:**
- `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI Foundry project endpoint
- `AZURE_OPENAI_DEPLOYMENT_NAME`: Your GPT-4/GPT-4o deployment name

In [ ]:
import os
import sys
import json
import logging
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Suppress verbose logging
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)
logging.getLogger("azure.identity").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Load environment variables
load_dotenv("../.env")

# Add utils to path
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), 'utils'))

# Verify environment variables
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"]
missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️  Warning: Missing environment variables: {missing}")
else:
    print("✅ Environment variables loaded successfully")
    print("✅ MCP SDK imported successfully")

### 1.3: Initialize Azure AI Client

Create Azure AI Project client and agent manager.

In [ ]:
# Get endpoint from environment
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env")

# Initialize client with Azure credential (following official SDK documentation)
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print("✅ Azure AI client initialized")
print(f"   Project endpoint: {endpoint}")

---

## Part 2: SSE MCP Server Integration

Connect to a remote MCP server using Server-Sent Events (SSE) transport.

### SSE Server Connection

SSE is used for remote MCP servers accessible over HTTP.

**Advantages:**
- Remote server access
- Cloud deployments
- Shared services
- Scalable architecture

In [ ]:
from mcp.client.sse import sse_client
from mcp.client.streamable_http import streamablehttp_client

async def connect_to_mcp_server_http(server_url: str, transport: str = "sse", headers: dict = None):
    """
    Connect to remote MCP server using HTTP-based transport.
    
    Args:
        server_url: URL of the MCP server
        transport: Transport type ("sse" or "streamable-http")
        headers: Optional headers (e.g., for authentication)
    """
    
    try:
        headers = headers or {}
        
        if transport.lower() in ["streamable-http", "stream-sse"]:
            # Streamable HTTP transport
            async with streamablehttp_client(server_url, headers) as (read, write, _):
                async with ClientSession(read, write) as session:
                    await session.initialize()
                    tools_result = await session.list_tools()
                    
                    print(f"✅ Connected to {server_url} (Streamable HTTP)")
                    print(f"   Tools: {len(tools_result.tools)}")
                    for tool in tools_result.tools:
                        print(f"      • {tool.name}: {tool.description}")
                    
                    return session, tools_result.tools
        else:
            # SSE (Server-Sent Events) transport
            async with sse_client(server_url, headers) as (read, write):
                async with ClientSession(read, write) as session:
                    await session.initialize()
                    tools_result = await session.list_tools()
                    
                    print(f"✅ Connected to {server_url} (SSE)")
                    print(f"   Tools: {len(tools_result.tools)}")
                    for tool in tools_result.tools:
                        print(f"      • {tool.name}: {tool.description}")
                    
                    return session, tools_result.tools
                    
    except Exception as e:
        print(f"❌ Connection failed: {type(e).__name__}: {str(e)}")
        print(f"💡 Tip: Use direct SDK integration (Part 7-8) for production")
        return None, []

# Example: Try to connect to an MCP server
server_url = "https://learn.microsoft.com/api/mcp"
transport_type = "stream-sse"  # or "sse" or "streamable-http"

session, tools = await connect_to_mcp_server_http(server_url, transport=transport_type)

if not session:
    print("\n⚠️  Manual SSE connection requires a real, accessible MCP server")
    print("📌 RECOMMENDED: Use direct SDK integration (Part 7-8)")


---

## Part 3: Complete MCP Integration Example

Complete example using direct MCP integration with Azure AI Projects SDK.

### Direct Integration Workflow

1. Define MCP server tool with type "mcp" (not "mcp_server")
2. Create agent with MCP tool
3. Create thread and post message
4. Run agent (SDK handles MCP calls automatically)
5. Read assistant response

In [ ]:
import time
from azure.ai.agents.models import (
    McpTool,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
)

### Step 1: Initialize MCP Tool

In [ ]:
# Initialize MCP tool
mcp_tool = McpTool(
    server_label="microsoft_docs",
    server_url="https://learn.microsoft.com/api/mcp",
    allowed_tools=[],  # Empty = all tools allowed
)

print(f"✅ MCP tool initialized: {mcp_tool.server_label}")
print(f"   Server URL: {mcp_tool.server_url}")
print(f"   Allowed tools: {mcp_tool.allowed_tools or 'All'}")

# Optional: Set approval mode
# mcp_tool.set_approval_mode("never")  # Disable approval requirement

### Step 2: Create Agent with MCP Tool

In [ ]:
# Create agent with MCP tool
agent = project_client.agents.create_agent(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    name="Microsoft Docs Assistant",
    instructions="""You are a helpful assistant that answers questions using official Microsoft documentation.
Use the MCP tools to search for accurate, up-to-date information and provide clear answers.""",
    tools=mcp_tool.definitions,  # Use mcp_tool.definitions
)

print(f"✅ Agent created: {agent.id}")

### Step 3: Create Thread and Post Message

In [ ]:
# Create thread
thread = project_client.agents.threads.create()
print(f"✅ Thread created: {thread.id}")

# Post message
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the latest REST API for Azure OpenAI deployments."
)
print(f"✅ Message posted")

### Step 4: Run Agent with MCP Tool Approval

In [ ]:
# Create run with MCP tool resources
run = project_client.agents.runs.create(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_resources=mcp_tool.resources
)
print(f"🚀 Run created: {run.id}")

# Poll run status and handle MCP tool approval
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    # Handle MCP tool approval requests
    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls

        if not tool_calls:
            print("   ⚠️  No tool calls - cancelling")
            project_client.agents.runs.cancel(
                thread_id=thread.id, run_id=run.id)
            break

        # Approve MCP tool calls
        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                print(f"   🔐 Approving: {tool_call.name}")
                tool_approvals.append(
                    ToolApproval(
                        tool_call_id=tool_call.id,
                        approve=True,
                        headers=mcp_tool.headers,
                    )
                )

        # Submit approvals
        if tool_approvals:
            project_client.agents.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_approvals=tool_approvals
            )

    print(f"   Status: {run.status}")

print(f"✅ Run completed: {run.status}")

if run.status == "failed":
    print(f"   Error: {run.last_error}")

### Step 5: Read Assistant Response

In [ ]:
# Read assistant response
messages = project_client.agents.messages.list(thread_id=thread.id)

for msg in messages:
    if msg.role == "assistant" and msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"🤖 Assistant: {last_text.text.value}")

---

## Summary and Best Practices

### Key Takeaways

1. **MCP Integration**: Use `McpTool` class for seamless integration with Azure AI agents
2. **5-Step Workflow**: Initialize tool → Create agent → Create thread → Run with approval → Read response
3. **Approval Handling**: Handle `RequiredMcpToolCall` in the polling loop for tool execution
4. **Production Ready**: SDK handles connections, tool discovery, and execution automatically

### Best Practices

**Configuration:**
- Use unique `server_label` for each MCP server
- Specify `allowed_tools` to limit tool access
- Include authentication in headers if required
- Use HTTPS URLs for production servers

**Error Handling:**
- Check run status and log `run.last_error`
- Implement retry logic for transient failures
- Handle authentication errors appropriately

**Security:**
- Store tokens in environment variables
- Use HTTPS for all connections
- Audit tool execution logs

**Testing:**
- Test MCP server independently first
- Verify tool availability before agent creation
- Validate error handling scenarios

### Production Considerations

- Deploy MCP servers as scalable services (Azure Container Apps, AKS)
- Monitor server response times and availability
- Set up alerts for failures
- Scale servers independently
- Rotate authentication tokens regularly

### Additional Resources

- [Azure AI Agents SDK](https://learn.microsoft.com/python/api/overview/azure/ai-projects-readme)
- [Model Context Protocol Specification](https://spec.modelcontextprotocol.io/)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)